In [1]:
import base64
import datetime
import requests
import pandas as pd
from config import clientID, clientSecret, redirectURI

In [2]:
clientCreds = f"{clientID}:{clientSecret}"
clientCredsB64 = base64.b64encode(clientCreds.encode())
clientCredsB64

b'YjIyZGNiMWYyZGM4NDc2Njg5Zjg3ZmVhNjg3YjEyNTk6NDRjYzlmMjgzMmZkNDAxMTlkZDI5ZDVhYmFiYzZlMWE='

In [3]:
tokenUrl = 'https://accounts.spotify.com/api/token'

method = "POST"
# POST
authResponse = requests.post(tokenUrl, {
    'grant_type': 'client_credentials',
    'client_id': clientID,
    'client_secret': clientSecret,
})

# convert the response to JSON
authResponseData = authResponse.json()

# save the access token
accessToken = authResponseData['access_token']
accessToken

In [6]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=accessToken)
}

# base URL of all Spotify API endpoints
baseUrl = 'https://api.spotify.com/v1/'

In [7]:
# Testing API call
# Track ID from the URI
trackId = '6y0igZArWVi6Iz0rj35c1Y'

# actual GET request with proper header
req = requests.get(baseUrl + 'audio-features/' + trackId, headers=headers)

req = req.json()
req

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [ ]:
# Led Zepplin test
artistId = '36QJpDe2go2KgaRleHCDTp'

# pull all artists albums
req = requests.get(baseUrl + 'artists/' + artistId + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
reqLedZepplin = req.json()

In [ ]:
for album in reqLedZepplin['items']:
    print(album['name'], ' --- ', album['release_date'])

In [ ]:
albums = [] # to keep track of duplicates

# Empty dataframe for later
df = pd.DataFrame(columns=['track_name', 'album_name', 'short_album_name', 'release_date', 'album_id'])

# loop over albums and get all tracks
for album in reqLedZepplin['items']:
    albumName = album['name']

    # here's a hacky way to skip over albums we've already grabbed
    trimName = albumName.split('(')[0].strip()
    if trimName.upper() in albums or int(album['release_date'][:4]) > 1983:
        continue
    albums.append(trimName.upper()) # use upper() to standardize
    
    # this takes a few seconds so let's keep track of progress    
    print(albumName)
    
    # pull all tracks from this album
    req = requests.get(baseUrl + 'albums/' + album['id'] + '/tracks', 
        headers=headers)
    tracks = req.json()['items']
    
    for track in tracks:
        # get audio features (key, liveness, danceability, ...)
        f = requests.get(baseUrl + 'audio-features/' + track['id'], 
            headers=headers)
        f = f.json()
        
        # combine with album info
        df.append({
            'track_name': track['name'],
            'album_name': albumName,
            'short_album_name': trimName,
            'release_date': album['release_date'],
            'album_id': album['id']
        })
    